# Protein Charge and Isoelectric Point Calculator

**Author:** Marilina Cathcarth
  - Email: mcathcarth@gmail.com
  - GitHub: [mcathcarth](https://github.com/mcathcarth)
    
**Version:** 1.1
    
**Date:** Jun 10, 2024
    
**Description:** This script calculates the net charge and isoelectric point (pI) of proteins based on their sequences extracted from PDB files. The net charge of each protein is computed over a specified pH range, and the data is saved and visualized.

## Dependencies

The script uses the following Python libraries:

- [Biopython](https://biopython.org/) (Bio) - Licensed under the Biopython License Agreement (BPLA)
- [NumPy](https://numpy.org/) - Licensed under the BSD License
- [Matplotlib](https://matplotlib.org/) - Licensed under the BSD License
- [requests](https://docs.python-requests.org/en/latest/) - Licensed under the Apache License 2.0

## Usage Instructions

1.  **Open the Colab Notebook:** Open the provided Colab notebook file.
    
2.  **Modify Input Parameters:**
    
    *   Update the `pdb_codes` list with the [PDB](https://www.rcsb.org/) codes of the proteins you want to analyze.
        
    *   Optionally, provide user names (`usr_names`) and number of chains (`usr_nc`) if needed.
        
    *   Set the parameters for `pH_start`, `pH_end`, `pH_step`, and `target_pH` as per your requirements.
        
3.  **Run Each Cell Sequentially:** Ensure that you run each cell in order, as some cells depend on the outputs or definitions from previous cells.
    
4.  **View Results:** After running the final cell, view the output which includes the isoelectric point (pI) and the net charge of each protein at the target pH. Additionally, a plot showing the net charge vs. pH will be generated and saved.


### Example Parameters

*   `pdb_codes`: A list of PDB codes for the proteins to analyze (e.g., ["4F5S", "1CF3", "1OVA"]).
    
*   `usr_names`: Optional list of user-provided names for the proteins.
    
*   `usr_nc`: Optional list of the number of chains to consider for each protein.
    
*   `pH_start`: Starting pH value for the range (e.g., 3.0).
    
*   `pH_end`: Ending pH value for the range (e.g., 10.0).
    
*   `pH_step`: Step size for the pH range (e.g., 0.1).
    
*   `target_pH`: Specific pH value at which to calculate the net charge (e.g., 7.4).

# Importing necessary modules

In [ ]:
# Installation of Biopython
!pip install Biopython

# Importing the necessary modules
import csv
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.PDB import PDBParser, Selection
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import requests
import warnings
from Bio import BiopythonWarning
import os

# Suppress Biopython warnings
warnings.simplefilter('ignore', BiopythonWarning)

# Input Parameters and Protein Sequences

Here, you'll input the parameters such as pH range and provide the protein codes.

In [ ]:
# 1. Enter the PDB codes of the proteins here.
#    You can find PDB codes at https://www.rcsb.org/
#    Example: ["4F5S", "1CF3", "1OVA"]

pdb_codes = [
    "",   # PDB code for protein 1
    "",   # PDB code for protein 2
    "",   # PDB code for protein 3
    # Add more sequences as needed
]

# 2. pH Parameters
#    Adjust the pH range and step according to your needs.

pH_start = 1      # Starting pH
pH_end = 13       # Ending pH
pH_step = 0.2     # pH increment
target_pH = 7.4   # Target pH for specific analysis


# --------------- Optional --------------- #

# 3. Optional: Custom names for the proteins (in the same order as the PDB codes)
#    If you prefer custom names, provide them here. Leave the list empty to use default names (protein_names = []).
#    Example: ["BSA", "GOx", "OVA"]

protein_names = [
    "",   # Custom name for protein 1
    "",   # Custom name for protein 2
    "",   # Custom name for protein 3
    # Add more names as needed
]

# 4. Enter the number of chains to analyze for each protein.
#    - To analyze all chains, enter 0.
#    - To analyze only the first chain, enter 1.
#    - To analyze multiple chains, enter the number of chains to include (e.g., 2 for the first two chains).
#    - If a protein has fewer chains than specified here, all available chains will be analyzed.

protein_chains = [
    0,  # Number of chains to analyze for protein 1
    0,  # Number of chains to analyze for protein 2
    0,  # Number of chains to analyze for protein 3
    # Add more as needed
]

# Note: Complete the required data in the lists above before running the analysis.
# Now you can proceed with the rest of the code for protein analysis.

# Functions Definition

In [ ]:
# Define global variables

# pKa values
pKa = {
    'R': 12.5, 'D': 3.5, 'C': 6.8, 'E': 4.2, 'H': 6.6, 'K': 10.5, 'Y': 10.3,
    'Nt': 7.7, 'Ct': 3.3, 'η': 3.8
}

# 3 letter code of amino acids
three_to_one = {
    'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C',
    'GLU': 'E', 'GLN': 'Q', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I',
    'LEU': 'L', 'LYS': 'K', 'MET': 'M', 'PHE': 'F', 'PRO': 'P',
    'SER': 'S', 'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V'
}

# Function to calculate protein charges and save data
def calculate_protein_charges_pI_and_save(pdb_codes, pH_start, pH_end, pH_step, target_pH, usr_names, usr_nc):
    """
    Calculate charges of proteins for a range of pH values and save the data to a CSV file.

    This function calculates the charges of proteins at different pH values using the given PDB codes.
    It then saves the calculated charges to a CSV file named 'net-charge-vs-pH.tsv' and plots the data.

    Parameters:
        pdb_codes (list of str): List of PDB codes for the proteins.
        pH_start (float): Starting pH value for the calculation.
        pH_end (float): Ending pH value for the calculation.
        pH_step (float): Step size between pH values.
        target_pH (float): The pH value at which to calculate the net charge.
        usr_names (list of str, optional): List of names corresponding to the proteins. Defaults to None.
        usr_nc (list of int, optional): The number of chains to analyze. Defaults to None.

    Returns:
        None

    Saves:
        net-charge-vs-pH.tsv: TSV file containing the calculated charges for each protein at different pH values.
        net-charge-vs-pH_plot.png: Plot of protein charges versus pH.
    """
    # Initialize lists
    usr_names = usr_names or [None] * len(pdb_codes)
    usr_nc = usr_nc or [0] * len(pdb_codes)

    protein_sequences = []
    protein_names = []
    protein_chains = []

    for code, name, chains in zip(pdb_codes, usr_names, usr_nc):
        # Download the file
        download_pdb_file(code)
        file_name = f"{code}.pdb"
        # Extract information from the file
        sequence, extracted_name, num_chains = extract_sequence_name_and_chains(file_name, chains, usr_name=name)
        # Add the information to the lists
        protein_sequences.append(sequence)
        protein_names.append(name if name else extracted_name)
        protein_chains.append(num_chains)

    print("\n----- Protein information -----\n")

    # Get pI and Q at pH
    pI_data = []
    target_charge_data = []

    for sequence, name, chains in zip(protein_sequences, protein_names, protein_chains):
        pI = calculate_protein_pI(sequence, chains)
        pI_data.append((name, pI))
        charge_at_target_pH = calculate_protein_charge(sequence, chains, target_pH)
        target_charge_data.append((name, charge_at_target_pH))

        # Print protein information
        print(f"For protein {name}:")
        #print(f"Full sequence: {sequence}")
        print(f"- Isoelectric Point (pI): {pI:.2f}")
        print(f"- Charge at pH {target_pH}: {charge_at_target_pH:.2f}")
        print()  # Empty line for better readability

    pH_range = np.arange(pH_start, pH_end + pH_step, pH_step)
    output_data = []

    # Define the size of the figure
    plt.figure(figsize=(10, 6))

    for sequence, name, chains in zip(protein_sequences, protein_names, protein_chains):
        charges = [calculate_protein_charge(sequence, chains, pH) for pH in pH_range]
        output_data.append([name, charges])

        # Plotting
        plt.plot(pH_range, charges, label=name)

    plt.xlabel('pH')
    plt.ylabel('Net Charge')
    plt.title('Net Charge vs. pH')
    plt.legend()
    plt.grid(True)
    plt.savefig('net_charge_vs_pH.png')
    plt.show()

    # Save data to CSV
    with open("net-charge-vs-pH.tsv", mode="w", newline="") as file:
        writer = csv.writer(file, delimiter='\t')
        writer.writerow(["pH"] + [name for name, _ in output_data])
        for i, pH in enumerate(pH_range):
            row = [pH] + [charges[i] for _, charges in output_data]
            writer.writerow(row)

    print("Data saved to net-charge-vs-pH.tsv")

# Function to download a PDB file
def download_pdb_file(protein_code):
    """
    Downloads a PDB file given a protein code.

    Args:
        protein_code (str): The code of the protein.

    Returns:
        None
    """
    url = f"https://files.rcsb.org/download/{protein_code}.pdb"
    file_name = f"{protein_code}.pdb"
    response = requests.get(url)
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f"File {file_name} downloaded successfully!")

# Function to extract sequences, names and chains
def extract_sequence_name_and_chains(pdb_file, usr_nc=None, usr_name=None):
    """
    Extracts amino acid sequences, protein names, and the number of chains from a PDB file.

    Args:
        pdb_file (str): The path to the PDB file.
        usr_nc (int, optional): The number of chains to analyze. If not provided or 0, all chains will be analyzed.
        usr_name (str, optional): The name of the protein. If not provided, it will be extracted from the PDB file.

    Returns:
        tuple: A tuple containing the concatenated amino acid sequence, protein name, and the number of chains in the PDB file.
    """

    protein_info, chains_info = get_protein_info(pdb_file)

    final_protein_name = usr_name if usr_name else protein_info[0]
    final_protein_name = final_protein_name if final_protein_name else protein_info[1]

    num_chains = len(chains_info)

    #sequences = get_seqres_sequence(pdb_file)
    sequences = get_ca_sequences(pdb_file)

    modified_sequences = modify_sequences_with_disulfide_and_heme(sequences, pdb_file)

    if usr_nc and usr_nc < num_chains:
        selected_chains = usr_nc
    else:
        selected_chains = num_chains

    final_sequence = "".join(modified_sequences[:selected_chains])

    return final_sequence, final_protein_name, num_chains

# Function to obtain protein information from DBREF records
def get_protein_info(pdb_file):
    """
    Obtains protein information from DBREF records in a PDB file.

    Args:
        pdb_file (str): The path to the PDB file.

    Returns:
        tuple: A tuple containing a list of protein information and a dictionary of chain information.
    """
    protein_info = []
    chain_info = {}
    with open(pdb_file, 'r') as file:
        for line in file:
            if line.startswith("DBREF"):
                parts = line.split()
                pdb_id = parts[1]
                chain_id = parts[2]
                seqBegin = parts[3]
                seqEnd = parts[4]
                #db_name = parts[5]
                #uniprot_id = parts[6]
                protein_name = parts[7]
                dbseqBegin = parts[8]
                dbseqEnd = parts[9]
                protein_info = [protein_name, pdb_id]
                chain_info[chain_id] = (pdb_id, seqBegin, seqEnd, protein_name, dbseqBegin, dbseqEnd)
    return protein_info, chain_info

# Function to obtain sequences from SEQRES records
def get_seqres_sequence(pdb_file):
    """
    Extracts sequences from SEQRES records in a PDB file.

    Args:
        pdb_file (str): The path to the PDB file.

    Returns:
        list: A list of sequences for each chain in the PDB file.
    """
    sequences = []
    num_chains = 0
    with open(pdb_file, "r") as file:
        for record in SeqIO.parse(file, "pdb-seqres"):
            num_chains += 1
            sequence = str(record.seq)
            chain_id = record.annotations['chain']
            sequences.append(sequence)
    return sequences

# Function to obtain sequences from CA atoms
def get_ca_sequences(pdb_file):
    """
    Extracts sequences from CA atoms in a PDB file.

    Args:
        pdb_file (str): The path to the PDB file.

    Returns:
        list: A list of sequences for each chain in the PDB file.
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('X', pdb_file)

    ca_sequences = {}
    for model in structure:
        for chain in model:
            chain_id = chain.get_id()
            if chain_id not in ca_sequences:
                ca_sequences[chain_id] = []
            for residue in chain:
                if residue.has_id('CA'):
                    resname = residue.get_resname()
                    ca_sequences[chain_id].append(three_to_one.get(resname, 'X'))  # Default to 'X' if the residue is unknown
    sequences = [''.join(seq) for seq in ca_sequences.values()]
    return sequences

# Function to modify sequences with disulfide bridges and heme groups
def modify_sequences_with_disulfide_and_heme(sequences, pdb_file):
    """
    Modifies amino acid sequences to replace cysteines involved in disulfide bridges with 'B'
    and adds 'n' to the end of sequences if they contain heme groups.

    Args:
        sequences (list): A list of amino acid sequences.
        pdb_file (str): The path to the PDB file.

    Returns:
        list: A list of modified sequences for each chain in the PDB file.
    """
    residues_with_disulfide = detect_disulfide_bridges(pdb_file)
    #has_heme, chains_with_heme = check_for_heme_and_chains(pdb_file)

    parser = PDBParser()
    structure = parser.get_structure("protein", pdb_file)
    modified_sequences = []

    for sequence, chain in zip(sequences, structure[0]):
        chain_seq = list(sequence)
        residue_mapping = {residue.get_full_id()[3][1]: residue for residue in chain.get_residues()}

        for residue in residues_with_disulfide:
            try:
                res_index = residue_mapping[residue.get_full_id()[3][1]].get_full_id()[3][1] - 1
                chain_seq[res_index] = "B"
            except KeyError:
                pass

        #if chain.id in chains_with_heme:
        #    chain_seq.append('η')

        modified_sequences.append("".join(chain_seq))

    return modified_sequences

  # Function to detect disulfide bridges
def detect_disulfide_bridges(pdb_file):
    """
    Detects disulfide bridges in a PDB file.
    Checks if a cysteine residue is involved in a disulfide bridge.

    Args:
        pdb_file (str): The path to the PDB file.

    Returns:
        list: A list of residues involved in disulfide bridges.
    """
    parser = PDBParser()
    structure = parser.get_structure("protein", pdb_file)
    model = structure[0]

    residues_with_disulfide = []

    for chain in model:
        cys_residues = Selection.unfold_entities(chain, 'R')
        cys_residues = [res for res in cys_residues if res.get_resname() == 'CYS']

        for i, res1 in enumerate(cys_residues):
            for res2 in cys_residues[i + 1:]:
                try:
                    atom1, atom2 = res1['SG'], res2['SG']
                except KeyError:
                    continue

                if atom1 - atom2 < 2.3:
                    residues_with_disulfide.extend([res1, res2])

    return list(set(residues_with_disulfide))

# Function to check heme groups and identify chains
def check_for_heme_and_chains(pdb_file):
    """
    Checks if a PDB file contains a heme group and identifies the chains containing the heme groups.

    Args:
        pdb_file (str): The path to the PDB file.

    Returns:
        tuple: A tuple containing a boolean indicating if a heme group is present and a list of chains containing the heme groups.
    """
    parser = PDBParser()
    structure = parser.get_structure("protein", pdb_file)
    chains_with_heme = []
    for model in structure:
        for chain in model:
            has_heme = False
            for residue in chain:
                if residue.resname == "HEM" or residue.resname == "HEC":
                    has_heme = True
                    break
            if has_heme:
                chains_with_heme.append(chain.id)

    return len(chains_with_heme) > 0, chains_with_heme

# Function to calculate the net charge of a protein at a given pH
def calculate_protein_charge(sequence, nc, pH):
    """
    Calculate the net charge of a protein at a given pH.

    This function calculates the net charge of a protein sequence at a specific pH based on the pKa values
    of its constituent amino acids.

    Parameters:
        sequence (str): The amino acid sequence of the protein.
        nc (int): The number of chains in the protein.
        pH (float): The pH value at which to calculate the net charge.

    Returns:
        float: The net charge of the protein at the given pH.
    """
    # Count charged amino acids
    R = sequence.count('R')  # Arginine (Arg) +
    D = sequence.count('D')  # Aspartate (Asp) -
    C = sequence.count('C')  # Cysteine (Cys) -
    E = sequence.count('E')  # Glutamate (Glu) -
    H = sequence.count('H')  # Histidine (His) +
    K = sequence.count('K')  # Lysine (Lys) +
    Y = sequence.count('Y')  # Tyrosine (Tyr) -
    #η = sequence.count('η')  # Heme group (Hem) -

    η=0
    # Calculate charge at a specific pH
    Q = nc / (1 + 10 ** (pH - pKa['Nt'])) - nc / (1 + 10 ** (-(pH - pKa['Ct']))) - \
        D / (1 + 10 ** (-(pH - pKa['D']))) - C / (1 + 10 ** (-(pH - pKa['C']))) - E / (1 + 10 ** (-(pH - pKa['E']))) - Y / (1 + 10 ** (-(pH - pKa['Y']))) - \
        2 * η / (1 + 10 ** (-(pH - pKa['η']))) + R / (1 + 10 ** (pH - pKa['R'])) +  H / (1 + 10 ** (pH - pKa['H'])) + K / (1 + 10 ** (pH - pKa['K']))

    return Q

# Function to calculate the isoelectric point (pI) of a protein
def calculate_protein_pI(sequence, nc):
    """
    Calculate the isoelectric point (pI) of a protein.

    This function calculates the isoelectric point (pI) of a protein sequence based on the pH range where
    the net charge of the protein is zero.

    Parameters:
        sequence (str): The amino acid sequence of the protein.
        nc (int): The number of chains in the protein.

    Returns:
        float: The isoelectric point (pI) of the protein.
    """
    pH_range = [0.1 + i * 0.001 for i in range(14000)]  # pH range for pI calculation
    Q_pI = [calculate_protein_charge(sequence, nc, pH) for pH in pH_range]

    for i in range(len(Q_pI) - 1):
        if Q_pI[i] * Q_pI[i + 1] <= 0:
            pI = 0.5 * (pH_range[i] + pH_range[i + 1])
            return pI

    return None  # If pI is not found

# Biopython #

# Function to calculate the net charge of a protein at a given pH
def calculate_protein_charge_bio(sequence, pH):
    """
    Calculate the net charge of a protein at a given pH.

    This function calculates the net charge of a protein sequence at a specific pH based on the pKa values
    of its constituent amino acids.

    Parameters:
        sequence (str): The amino acid sequence of the protein.
        pH (float): The pH value at which to calculate the net charge.

    Returns:
        float: The net charge of the protein at the given pH.
    """
    # Create a ProteinAnalysis object with the sequence
    protein = ProteinAnalysis(sequence)
    # Calculate the charge of the protein at the target pH
    Q = protein.charge_at_pH(pH)

    return Q

# Function to calculate the isoelectric point (pI) of a protein
def calculate_protein_pI_bio(sequence):
    """
    Calculate the isoelectric point (pI) of a protein.

    This function calculates the isoelectric point (pI) of a protein sequence based on the pH range where
    the net charge of the protein is zero.

    Parameters:
        sequence (str): The amino acid sequence of the protein.

    Returns:
        float: The isoelectric point (pI) of the protein.
    """
    # Create a ProteinAnalysis object with the sequence
    protein = ProteinAnalysis(sequence)
    # Calculate isoelectric point (pI) of the protein
    pI = protein.isoelectric_point()

    return pI

# Execution

Here's where you call the function with the provided parameters.

In [ ]:
calculate_protein_charges_pI_and_save(pdb_codes, pH_start, pH_end, pH_step, target_pH, protein_names, protein_chains)